In [42]:
import requests
import json
import pandas as pd

In [43]:
'''getQuery function takes a url request and header dict, then returns a formatted json file'''
def getQuery(url, headers):
    # Get get request for json 
    r = requests.get(url, headers=headers)
    # use json loader to format output
    json_data = json.loads(r.text)
    # return the data and when that data was queried
    return json_data

'''formatQuery takes a json_data set then formats it according to the runeliteAPI timeseries data then returns the results'''
def formatQuery(json_data, itemKeys): 
    samples = []
    for itemID in itemKeys['id']:
        strID = str(itemID)
        if strID in json_data['data']:
            # Get name based on itemID
            itemName = itemKeys[itemKeys['id'] == itemID]['name'].iloc[0]
            # Calc the margin based on high and low
            margin = json_data['data'][strID]['high'] - json_data['data'][strID]['low']
            
            # Get the tax adjusted margin. GE tax cap is 5mil
            tax = min((json_data['data'][strID]['high'] * 0.01), 5000000)
            marginAdjusted = (json_data['data'][strID]['high'] - tax) - json_data['data'][strID]['low']
            entry = (
                itemID,
                itemName,
                json_data['data'][strID]['high'],
                json_data['data'][strID]['highTime'],
                json_data['data'][strID]['low'],
                json_data['data'][strID]['lowTime'],
                margin,
                marginAdjusted 
            )
            samples.append(entry)
    return samples

In [44]:
'''Define needed variables'''
# API key used to access summary jsons
runeliteAPI = "https://prices.runescape.wiki/api/v1/osrs/latest"

print(f"Query being used: {runeliteAPI}")

# Get item key data to match ids to items
itemKeys = pd.read_csv("data/item_keys.csv")

# User agent tag used to let wiki know why I am using their API. 
userAgent = {
    'User-Agent': 'Machine Learning Practice',
    'From': 'sliverwall (discord ID)'  # discord ID
}

# Define where data will be written to
fileName = f"runelite_data_margins"
outputDir = "data/"
fileExe = ".csv"
filePath = outputDir + fileName + fileExe

# Define headers for output file
colNames = ["itemID", "name", "high", "highTime", "low", "lowTime", "margin", "marginAdjusted"]

Query being used: https://prices.runescape.wiki/api/v1/osrs/latest


In [46]:
'''Main function'''
# Query for all known margins from /latests
jsonData = getQuery(runeliteAPI, userAgent)

# format into a sample list
samples = formatQuery(jsonData, itemKeys)

# Create df for the margin data
marginData = pd.DataFrame(samples, columns=colNames)


'''Merge limits to this table'''
# Read in item keys to match limits to given items
itemKeys = pd.read_csv("data/item_keys.csv")

filteredItemKeys = itemKeys[['id', 'limit']]

# Now merge
merged_df = pd.merge(
    marginData, filteredItemKeys,
    left_on='itemID',
    right_on='id',
    how='left'
)
# Drop the extra id column
merged_df = merged_df.drop(columns='id')

# Add a potential profited based on marginAdjusted * limit
merged_df['potentialProfit'] = merged_df['marginAdjusted'] * merged_df['limit']

# Sort column based on marginAdjusted in descending order
merged_df = merged_df.sort_values(by='marginAdjusted', ascending=False)

'''Write final output to a csv'''
# Write data to csv
merged_df.to_csv(filePath, index=False)

print(f"Margin data written to {filePath}")

Margin data written to data/runelite_data_margins.csv
